In [ ]:
import pandas as pd
import lux

In [ ]:
# Collecting basic usage statistics for Lux (For more information, see: https://tinyurl.com/logging-consent)
lux.logger = True # Remove this line if you do not want your interactions recorded

In [ ]:
df = pd.read_csv("https://github.com/lux-org/lux-datasets/blob/master/data/hpi.csv?raw=true")
lux.config.default_display = "lux"
df

In [ ]:
df["G10"]  = df["Country"].isin(["Belgium","Canada","France","Germany","Italy","Japan","Netherlands","United Kingdom","Switzerland","Sweden","United States"])

In [ ]:
from lux.vis.VisList import VisList

In [ ]:
intent = [lux.Clause("?",data_type="quantitative"),lux.Clause("G10")]
vlist = VisList(intent,df)

for vis in vlist:
    # Percentage Change Between G10 v.s. non-G10 countries 
    a = vis.data.iloc[0,1]
    b = vis.data.iloc[1,1]
    vis.score = (b-a)/a
vlist = vlist.topK(15)

In [ ]:
vlist

In [ ]:
def G10_mean_difference(input_df): #TODO: change input_df name? 
    intent = [lux.Clause("?",data_type="quantitative"),lux.Clause("G10")]
    vlist = VisList(intent,input_df)

    for vis in vlist:
        a = vis.data.iloc[0,1]
        b = vis.data.iloc[1,1]
        vis.score = (b-a)/a
    vlist = vlist.topK(15)
    return {"action":"G10", "description": "Percentage Change of Means Between G10 v.s. non-G10 countries", "collection": vlist}

In [ ]:
def is_G10_hpi_dataset(df):
    try: 
        return all(df.columns == ['HPIRank', 'Country', 'SubRegion', 'AverageLifeExpectancy',
       'AverageWellBeing', 'HappyLifeYears', 'Footprint',
       'InequalityOfOutcomes', 'InequalityAdjustedLifeExpectancy',
       'InequalityAdjustedWellbeing', 'HappyPlanetIndex', 'GDPPerCapita',
       'Population', 'G10'])
    except: 
        return False

In [ ]:
lux.register_action("G10", G10_mean_difference,is_G10_hpi_dataset)

Now we can look at the attributes that are most different across G10 and non-G10 countries. We find that G10 countries have higher GDPPerCapita, but also contribute to more Footprint.

In [ ]:
df

What if we look at countries above a certain GDP threshold? 

In [ ]:
df[df["GDPPerCapita"]>40000]

In [ ]:
lux.actions.__len__()
lux.actions.__getactions__()

In [ ]:
lux.actions.__getattr__("G10")

In [ ]:
lux.remove_action("G10")